In [ ]:
import csv
from collections import namedtuple
from getpass import getpass
from rich.console import Console
from rich.prompt import Prompt
from rich.panel import Panel
import os

console = Console()

ARQ_USUARIOS = 'usuarios.txt'
ARQ_SERVICOS = 'servicos.txt'

USUARIO_LOGADO = None

Usuario = namedtuple('Usuario', ['login', 'senha', 'tipo'])
Servico = namedtuple('Servico', ['codigo', 'nome', 'preco', 'quantidade'])

############################
# Função para criar arquivos com dados iniciais se não existirem
def criar_arquivos_iniciais():
    if not os.path.exists(ARQ_USUARIOS):
        usuarios_iniciais = [
            ['admin', 'admin123', 'admin'],
            ['tecnico1', 'tec123', 'tecnico'],
            ['tecnico2', 'tec234', 'tecnico'],
            ['funcionario', 'func123', 'tecnico'],
            ['cliente1', 'cli123', 'cliente'],
            ['cliente2', 'cli234', 'cliente'],
        ]
        with open(ARQ_USUARIOS, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(usuarios_iniciais)
        console.print("[green]Arquivo usuarios.txt criado com dados iniciais.[/green]")

    if not os.path.exists(ARQ_SERVICOS):
        servicos_iniciais = [
            [1, 'Troca de vela', 50.0, 10],
            [2, 'Limpeza de queimador', 80.0, 5],
            [3, 'Conserto de válvula', 100.0, 7],
            [4, 'Manutenção geral', 150.0, 3],
            [5, 'Substituição de termopar', 60.0, 12],
            [6, 'Ajuste do forno', 70.0, 6],
        ]
        with open(ARQ_SERVICOS, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(servicos_iniciais)
        console.print("[green]Arquivo servicos.txt criado com dados iniciais.[/green]")

############################
# Função para ler usuários do arquivo
def ler_usuarios():
    usuarios = {}
    with open(ARQ_USUARIOS, 'r', newline='') as f:
        reader = csv.reader(f)
        for login, senha, tipo in reader:
            usuarios[login] = Usuario(login, senha, tipo)
    return usuarios

# Função para salvar usuários em arquivo
def salvar_usuarios(usuarios):
    with open(ARQ_USUARIOS, 'w', newline='') as f:
        writer = csv.writer(f)
        for u in usuarios.values():
            writer.writerow([u.login, u.senha, u.tipo])

############################
# Função para ler serviços
def ler_servicos():
    servicos = []
    with open(ARQ_SERVICOS, 'r', newline='') as f:
        reader = csv.reader(f)
        for codigo, nome, preco, quantidade in reader:
            servicos.append(Servico(int(codigo), nome, float(preco), int(quantidade)))
    return servicos

# Função para salvar serviços
def salvar_servicos(servicos):
    with open(ARQ_SERVICOS, 'w', newline='') as f:
        writer = csv.writer(f)
        for s in servicos:
            writer.writerow([s.codigo, s.nome, s.preco, s.quantidade])

############################
# Login
def login(usuarios):
    global USUARIO_LOGADO
    console.print(Panel("[bold green]Login[/bold green]\nInsira suas credenciais", expand=False))
    login = Prompt.ask("Login")
    senha = getpass("Senha: ")
    user = usuarios.get(login)
    if user and user.senha == senha:
        USUARIO_LOGADO = user
        console.print(f"[green]Login realizado com sucesso! Bem-vindo(a), {user.login} ({user.tipo})[/green]")
        return True
    else:
        console.print("[red]Login ou senha incorretos.[/red]")
        return False

############################
# Menu principal
def menu_principal():
    while True:
        console.print(Panel(f"Bem-vindo ao sistema Reparar Fogão!\nUsuário: {USUARIO_LOGADO.login} ({USUARIO_LOGADO.tipo})", expand=False))
        console.print("1. Atualizar senha")
        console.print("2. Gerenciar serviços")
        if USUARIO_LOGADO.tipo == 'admin':
            console.print("3. Gerenciar usuários")
        console.print("0. Logout")

        opcoes = ['0','1','2']
        if USUARIO_LOGADO.tipo == 'admin':
            opcoes.append('3')

        escolha = Prompt.ask("Escolha", choices=opcoes)

        if escolha == '0':
            console.print("Logout realizado.")
            break
        elif escolha == '1':
            atualizar_senha()
        elif escolha == '2':
            menu_servicos()
        elif escolha == '3' and USUARIO_LOGADO.tipo == 'admin':
            menu_usuarios()

############################
# Atualizar senha
def atualizar_senha():
    global USUARIO_LOGADO
    console.print(Panel("Atualizar Senha", expand=False))
    nova_senha = getpass("Digite a nova senha: ")
    usuarios = ler_usuarios()
    user = usuarios.get(USUARIO_LOGADO.login)
    if user:
        usuarios[USUARIO_LOGADO.login] = Usuario(user.login, nova_senha, user.tipo)
        salvar_usuarios(usuarios)
        USUARIO_LOGADO = usuarios[USUARIO_LOGADO.login]
        console.print("[green]Senha atualizada com sucesso![/green]")
    else:
        console.print("[red]Erro ao atualizar senha.[/red]")

############################
# Menus e funções para serviços

def menu_servicos():
    servicos = ler_servicos()
    while True:
        console.print(Panel("Menu de Serviços", expand=False))
        console.print("1. Listar todos os serviços")
        console.print("2. Buscar serviço por nome")
        if USUARIO_LOGADO.tipo == 'admin':
            console.print("3. Cadastrar novo serviço")
            console.print("4. Editar serviço")
            console.print("5. Remover serviço")
            console.print("6. Listar serviços ordenados por nome")
            console.print("7. Listar serviços ordenados por preço")
            console.print("0. Voltar")
            escolha = Prompt.ask("Escolha", choices=[str(i) for i in range(8)])
        else:  # tecnico só algumas opções
            console.print("3. Editar serviço")
            console.print("0. Voltar")
            escolha = Prompt.ask("Escolha", choices=["0","1","2","3"])

        if escolha == '1':
            listar_servicos(servicos)
        elif escolha == '2':
            buscar_servico(servicos)
        elif escolha == '3':
            if USUARIO_LOGADO.tipo == 'admin':
                cadastrar_servico(servicos)
            else:
                editar_servico(servicos)
        elif escolha == '4' and USUARIO_LOGADO.tipo == 'admin':
            editar_servico(servicos)
        elif escolha == '5' and USUARIO_LOGADO.tipo == 'admin':
            remover_servico(servicos)
        elif escolha == '6' and USUARIO_LOGADO.tipo == 'admin':
            listar_ordenado_nome(servicos)
        elif escolha == '7' and USUARIO_LOGADO.tipo == 'admin':
            listar_ordenado_preco(servicos)
        elif escolha == '0':
            break
        else:
            console.print("[red]Opção inválida![/red]")

def listar_servicos(servicos):
    console.print(Panel("Lista de Serviços", expand=False))
    for s in servicos:
        console.print(f"{s.codigo} - {s.nome} | R${s.preco:.2f} | Quantidade: {s.quantidade}")

def buscar_servico(servicos):
    termo = Prompt.ask("Digite o nome ou parte do serviço")
    encontrados = [s for s in servicos if termo.lower() in s.nome.lower()]
    if encontrados:
        console.print(Panel(f"Serviços encontrados para '{termo}'", expand=False))
        for s in encontrados:
            console.print(f"{s.codigo} - {s.nome} | R${s.preco:.2f} | Quantidade: {s.quantidade}")
    else:
        console.print(f"[yellow]Nenhum serviço encontrado para '{termo}'[/yellow]")

def cadastrar_servico(servicos):
    console.print(Panel("Cadastrar Novo Serviço", expand=False))
    try:
        codigo = max([s.codigo for s in servicos]) + 1 if servicos else 1
    except ValueError:
        codigo = 1
    nome = Prompt.ask("Nome do serviço")
    preco = float(Prompt.ask("Preço do serviço", default="0"))
    quantidade = int(Prompt.ask("Quantidade disponível", default="0"))
    novo = Servico(codigo, nome, preco, quantidade)
    servicos.append(novo)
    salvar_servicos(servicos)
    console.print(f"[green]Serviço '{nome}' cadastrado com sucesso![/green]")

def editar_servico(servicos):
    console.print(Panel("Editar Serviço", expand=False))
    try:
        codigo = int(Prompt.ask("Código do serviço para editar"))
    except ValueError:
        console.print("[red]Código inválido[/red]")
        return
    for i, s in enumerate(servicos):
        if s.codigo == codigo:
            nome = Prompt.ask("Novo nome", default=s.nome)
            try:
                preco = float(Prompt.ask("Novo preço", default=str(s.preco)))
                quantidade = int(Prompt.ask("Nova quantidade", default=str(s.quantidade)))
            except ValueError:
                console.print("[red]Preço ou quantidade inválidos[/red]")
                return
            servicos[i] = Servico(codigo, nome, preco, quantidade)
            salvar_servicos(servicos)
            console.print("[green]Serviço atualizado com sucesso![/green]")
            return
    console.print("[yellow]Serviço não encontrado[/yellow]")

def remover_servico(servicos):
    console.print(Panel("Remover Serviço", expand=False))
    try:
        codigo = int(Prompt.ask("Código do serviço para remover"))
    except ValueError:
        console.print("[red]Código inválido[/red]")
        return
    for i, s in enumerate(servicos):
        if s.codigo == codigo:
            servicos.pop(i)
            salvar_servicos(servicos)
            console.print(f"[green]Serviço '{s.nome}' removido com sucesso![/green]")
            return
    console.print("[yellow]Serviço não encontrado[/yellow]")

def listar_ordenado_nome(servicos):
    ordenados = sorted(servicos, key=lambda s: s.nome.lower())
    console.print(Panel("Serviços ordenados por nome", expand=False))
    for s in ordenados:
        console.print(f"{s.codigo} - {s.nome} | R${s.preco:.2f} | Quantidade: {s.quantidade}")

def listar_ordenado_preco(servicos):
    ordenados = sorted(servicos, key=lambda s: s.preco)
    console.print(Panel("Serviços ordenados por preço", expand=False))
    for s in ordenados:
        console.print(f"{s.codigo} - {s.nome} | R${s.preco:.2f} | Quantidade: {s.quantidade}")

############################
# Menus e funções para usuários (só admin pode acessar)
def menu_usuarios():
    usuarios = ler_usuarios()
    while True:
        console.print(Panel("Gerenciar Usuários", expand=False))
        console.print("1. Listar usuários")
        console.print("2. Buscar usuário por login")
        console.print("3. Cadastrar usuário")
        console.print("4. Editar usuário")
        console.print("5. Remover usuário")
        console.print("0. Voltar")
        escolha = Prompt.ask("Escolha", choices=[str(i) for i in range(6)])

        if escolha == '1':
            listar_usuarios(usuarios)
        elif escolha == '2':
            buscar_usuario(usuarios)
        elif escolha == '3':
            usuarios = cadastrar_usuario(usuarios)
        elif escolha == '4':
            usuarios = editar_usuario(usuarios)
        elif escolha == '5':
            usuarios = remover_usuario(usuarios)
        elif escolha == '0':
            break

def listar_usuarios(usuarios):
    console.print(Panel("Lista de Usuários", expand=False))
    for u in usuarios.values():
        console.print(f"Login: {u.login} | Tipo: {u.tipo}")

def buscar_usuario(usuarios):
    login = Prompt.ask("Login do usuário")
    u = usuarios.get(login)
    if u:
        console.print(f"Login: {u.login} | Tipo: {u.tipo}")
    else:
        console.print(f"[yellow]Usuário '{login}' não encontrado[/yellow]")

def cadastrar_usuario(usuarios):
    console.print(Panel("Cadastrar Novo Usuário", expand=False))
    login = Prompt.ask("Login")
    if login in usuarios:
        console.print(f"[red]Usuário '{login}' já existe[/red]")
        return usuarios
    senha = getpass("Senha: ")
    tipo = Prompt.ask("Tipo (admin/tecnico/cliente)", choices=["admin","tecnico","cliente"])
    usuarios[login] = Usuario(login, senha, tipo)
    salvar_usuarios(usuarios)
    console.print(f"[green]Usuário '{login}' cadastrado com sucesso![/green]")
    return usuarios

def editar_usuario(usuarios):
    console.print(Panel("Editar Usuário", expand=False))
    login = Prompt.ask("Login do usuário a editar")
    u = usuarios.get(login)
    if not u:
        console.print(f"[yellow]Usuário '{login}' não encontrado[/yellow]")
        return usuarios
    senha = getpass("Nova senha (enter para manter atual): ")
    tipo = Prompt.ask("Novo tipo (admin/tecnico/cliente) (enter para manter atual)", choices=["admin","tecnico","cliente",""], default="")
    senha = senha if senha.strip() != '' else u.senha
    tipo = tipo if tipo.strip() != '' else u.tipo
    usuarios[login] = Usuario(login, senha, tipo)
    salvar_usuarios(usuarios)
    console.print(f"[green]Usuário '{login}' atualizado com sucesso![/green]")
    return usuarios

def remover_usuario(usuarios):
    console.print(Panel("Remover Usuário", expand=False))
    login = Prompt.ask("Login do usuário a remover")
    if login in usuarios:
        del usuarios[login]
        salvar_usuarios(usuarios)
        console.print(f"[green]Usuário '{login}' removido com sucesso![/green]")
    else:
        console.print(f"[yellow]Usuário '{login}' não encontrado[/yellow]")
    return usuarios

############################
# Função principal para rodar o sistema
def main():
    criar_arquivos_iniciais()
    usuarios = ler_usuarios()

    while True:
        console.print(Panel("Reparar Fogão - Menu Inicial", expand=False))
        console.print("1. Login")
        console.print("2. Cadastro (somente técnico)")
        console.print("3. Sair")
        escolha = Prompt.ask("Escolha", choices=["1","2","3"])

        if escolha == '1':
            if login(usuarios):
                menu_principal()
                usuarios = ler_usuarios()  # recarregar depois de alterações
        elif escolha == '2':
            # só técnico pode se cadastrar por aqui
            login_novo = Prompt.ask("Escolha um login para técnico")
            if login_novo in usuarios:
                console.print(f"[red]Login '{login_novo}' já existe![/red]")
            else:
                senha_nova = getpass("Senha: ")
                usuarios[login_novo] = Usuario(login_novo, senha_nova, 'tecnico')
                salvar_usuarios(usuarios)
                console.print(f"[green]Usuário técnico '{login_novo}' cadastrado com sucesso![/green]")
                usuarios = ler_usuarios()
        elif escolha == '3':
            console.print("[blue]Saindo do sistema. Até logo![/blue]")
            break

if __name__ == "__main__":
    main()

Arquivo usuarios.txt criado com dados iniciais.

Arquivo servicos.txt criado com dados iniciais.

╭──────────────────────────────╮
│ Reparar Fogão - Menu Inicial │
╰──────────────────────────────╯

1. Login

2. Cadastro (somente técnico)

...

5. Remover serviço

6. Listar serviços ordenados por nome

7. Listar serviços ordenados por preço

0. Voltar

Escolha [0/1/2/3/4/5/6/7]: